### 1. Import Modules

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, losses, callbacks, applications, models
import numpy as np
import matplotlib.pyplot as plt
import os, datetime

In [ ]:
print(os.getcwd())

### 2. Load Datasets

In [ ]:
PATH = f"{os.getcwd()}/Small-Dataset"

BATCH_SIZE = 27
IMG_SIZE = (160, 160)

dataset = tf.keras.utils.image_dataset_from_directory(PATH,
                                                        shuffle=True,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMG_SIZE)

### 3. Data Inspection

In [ ]:
class_names = dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### 4. Train/Test/Validation Split

In [ ]:
batches_train_test = tf.data.experimental.cardinality(dataset)
train_dataset = dataset.skip(batches_train_test//4)
test_dataset_before_split = dataset.take(batches_train_test//4)

batches_test_val = tf.data.experimental.cardinality(test_dataset_before_split)
test_dataset = test_dataset_before_split.take(batches_test_val//2)
validation_dataset = test_dataset_before_split.skip(batches_test_val//2)

print(len(dataset))
print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

### 5. Converting Tensorflow Dataset to Prefetch Dataset

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### 6. Creating a Sequential Model for Image Augmentation

In [ ]:
data_augmentation = keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

### 7. Visualizing Data Augmentation

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### 8. Data Normalization

In [ ]:
preprocess_input = applications.mobilenet_v2.preprocess_input

### 9. Construct the Transfer Learning Pipeline

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model.summary()

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
global_avg = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(len(class_names), activation="softmax")
inputs = keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_avg(x)
x = layers.Dropout(0.3)(x)
outputs = output_layer(x)
model = keras.Model(inputs=inputs, outputs=outputs)

### 10. Compile the Model

In [ ]:
optimizer = optimizers.Adam(learning_rate = 0.0001)
loss = losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
early_stopping = callbacks.EarlyStopping(patience=2)
PATH = os.getcwd()
logpath = os.path.join(PATH, 'tensorboard_log', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
datetime.datetime.now()
tb = callbacks.TensorBoard(logpath)

### 11. Model Training

In [ ]:
EPOCHS = 10
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, callbacks=[early_stopping, tb])

### 12. Evaluate the Model

In [ ]:
model.evaluate(test_dataset)

### 13. Model Fine Tuning

In [ ]:
base_model.trainable=True
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layers.trainable = False
base_model.summary()

### 14. Compile, Fine Tune and Train Again

In [ ]:
optimizer = optimizers.RMSprop(learning_rate=0.00001)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
fine_tune_epoch = 10
total_epoch = EPOCHS + fine_tune_epoch
history_fine = model.fit(train_dataset, validation_data=validation_dataset, epochs=total_epoch, initial_epoch=history.epoch[-1], callbacks=[tb, early_stopping])

### 15. Save Model

In [ ]:
if os.path.exists(f"{os.getcwd()}/runs") is False:
    os.makedir("runs")

num = 1
filename = f"model{num}.h5"
while os.path.exists(f"{os.getcwd()}/runs/{filename}"):
    num += 1
    filename = f"model{num}.h5"
model.save(f"{os.getcwd()}/runs/{filename}")